# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/andreacolavita/Documents/GitHub/project-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [15]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cassandra_project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [16]:
try:
    session.set_keyspace("cassandra_project")
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data:


### Question 1
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### Answer 1

The Question 1 expects Name of the artist, title of the song and length of the track based on sessionId and itemInSession.
As per NoSQL data modeling, 1-query-1-table, we need to know the query in advance which will be used to create the Cassandra table.

##### While creating the table, we choose the PRIMARY KEY with the addition of CLUSTERING COLUMN to uniquely identify each row based on the question that we need to answer. In this case our primary key will be the session_id (it is used to distribute the data among the nodes of the cluster) and our clustering column will be the itemInSession (it is used to sort the data). 

In [10]:
query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(artist varchar, song varchar, length float, session_id int, itemInSession int, PRIMARY KEY ((session_id), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Read the CSV file with the columns that we need for the query 1 and store it into the Cassandra table 

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_history (artist, song, length, session_id, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### We SELECT the Artist, Song and Lenght where sessionId = 602 and itemInSession = 2

In [17]:
query = "SELECT artist, song, length FROM music_app_history where session_id = 602 and itemInSession = 2"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Van Halen Best Of Both Worlds (Remastered Album Version) 289.384033203125


### Question 2
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Answer 2
The Question 2 expects Name of the artist, Title of the song, First Name and Last Name of the user for a given userId and sessionId. As per NoSQL data modeling, 1-query-1-table, we need to know the query in advance which will be used to create the Cassandra table.

##### While creating the table, we choose the PRIMARY KEY with the addition of CLUSTERING COLUMN to uniquely identify each row based on the question that we need to answer. In this case our primary key will be the user_id (it is used to distribute the data among the nodes of the cluster) and our clustering columns will be the session_id and itemInSession (it is used to sort the data). 


In [13]:
query = "CREATE TABLE IF NOT EXISTS music_app_users "
query = query + "(artist varchar, song varchar, firstname varchar, lastname varchar, session_id int, itemInSession int, user_id int, PRIMARY KEY ((user_id), session_id, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                                     

#### Read the CSV file with the columns that we need for the query 2 and store it into the Cassandra table 

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_users (artist, song, firstname, lastname, session_id, itemInSession, user_id)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[8]), int(line[3]), int(line[10])))

#### We SELECT the Artist, Song, First and Last Name where userId = 80 and sessionId = 602

In [18]:
query = "SELECT artist, song, firstname, lastname FROM music_app_users where user_id = 80 and session_id = 602"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Van Halen Best Of Both Worlds (Remastered Album Version) Tegan Levine
Magic Sam Call Me If You Need Me Tegan Levine
Edward Sharpe & The Magnetic Zeros Home Tegan Levine
Usher featuring will.i.am OMG Tegan Levine
Helen Reddy Candle On The Water Tegan Levine
Taylor Swift Our Song Tegan Levine
Sean Paul Baby Boy [feat. Beyonce] Tegan Levine
The Killers Human Tegan Levine


### Question 3 

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Answer 3 
The Question 3 expects First and Last names of the users which heared a given song. As per NoSQL data modeling, 1-query-1-table, we need to know the query in advance which will be used to create the Cassandra table.

##### While creating the table, we choose the PRIMARY KEY with the addition of CLUSTERING COLUMN to uniquely identify each row based on the question that we need to answer. In this case our primary key will be the song (it is used to distribute the data among the nodes of the cluster) and our clustering columns will be the session_id and itemInSession (it is used to sort the data). 

In [18]:
query = "CREATE TABLE IF NOT EXISTS music_app_songs "
query = query + "(firstname varchar, lastname varchar, session_id int, itemInSession int, user_id int, song varchar, PRIMARY KEY ((song), session_id, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_songs (firstname, lastname, session_id, itemInSession, user_id, song)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], int(line[8]), int(line[3]), int(line[10]), line[9]))

#### We SELECT the First and Last Name of th users which heard the song 'The Big Gundown'

In [19]:
query = "SELECT firstname, lastname FROM music_app_songs where song = 'The Big Gundown'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Tegan Levine
Ryan Smith


### Drop the tables before closing out the sessions

In [22]:
query = "DROP TABLE music_app_history"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_app_users"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_app_songs"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()